In [24]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import StackingClassifier, StackingRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import StratifiedKFold

* Data Preparations


In [25]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")


In [26]:
def transform(df):
    object_df = df.select_dtypes(include=['object'])
    object_columns = object_df.columns
    return pd.get_dummies(object_df, prefix=object_columns), object_columns

def connectData(df, transformed, columns, all_dummy_columns):
    return pd.concat([df.drop(columns, axis=1), transformed.reindex(columns=all_dummy_columns, fill_value=False)], axis=1)

def split(title, df):
    return df.drop(title, axis=1), df[title]

In [27]:
train_transformed_data, train_deserted_columns = transform(train_df.drop(["id", "CustomerId", "Surname"], axis=1))
test_transformed_data, test_deserted_columns = transform(test_df.drop(["id", "CustomerId", "Surname"], axis=1))
all_dummy_columns = set(train_transformed_data.columns).union(test_transformed_data.columns)

train_converted_df = connectData(train_df, train_transformed_data, train_deserted_columns, all_dummy_columns)
test_converted_df = connectData(test_df, test_transformed_data, test_deserted_columns, all_dummy_columns)

train_x, train_y = split("Exited", train_converted_df.drop(["id", "CustomerId", "Surname"], axis=1))
test_x = test_converted_df.drop(["id", "CustomerId", "Surname"], axis=1)

# Find The Best Params

In [28]:
classifier = xgb.XGBClassifier(n_estimators=2000)
classifier.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [34]:
machine_predicting_ans = classifier.predict_proba(test_x)[:, 1]

In [35]:
submitting = pd.read_csv("../sample_submission.csv")

In [36]:
submit_test = test_x.copy()
submit_test['id'] = test_df['id']
submit_test['Exited'] = machine_predicting_ans
final_df = submitting.drop("Exited", axis=1).merge(submit_test[["Exited", "id"]],on='id', how='left')

In [ ]:
final_df.to_csv("../../pedicted_data/XGBmoreEstimatorsSubmission_2.csv", index=False)